# Visual attention - Lab 1 - From the eye-tracker to saliency maps

Created by : Alexandre Bruckert / University of Nantes - alexandre.bruckert@univ-nantes.fr

Date : 2024

In this lab, we will learn how to process eye-tracking data as output by the eye-tracker in order to transform it into a fixation map, or a visual saliency map.

To this end, you will have to fill out the functions specified in this notebook. 

In [ ]:
%matplotlib inline

import os
import glob
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.ndimage import gaussian_filter

PATH_DATA = "path\to\the\dataset\directory"

# Experiment-dependant constants
RESO_X = 1280
RESO_Y = 1024
FACTOR_X = RESO_X/338
FACTOR_Y = RESO_Y/270

First, for each image, we want to gather the files from all observers output by the eye-tracker.
For this dataset, file structure is the following :  
IIRCCyN_IVC_Eyetracker_Berkeley_Database  
> Images  
> Eyetracker_Data  
>>Obs1  
>>.  
>>.     
>>ObsN  
>>> image_img.trn.1.png-obs_ObsN.txt  
>>>.  
>>>.  
>>> image_img.trn.N.png-obs_ObsN.txt  
>>> image_img.tst.1.png-obs_ObsN.txt  
>>>.  
>>>.  
>>> image_img.tst.N.png-obs_ObsN.txt


In [ ]:
def get_files_all_observers(dir_path, img_name):
    """
    List all eye-tracking data files related to a particular image.
    
    :param dir_path: str, the path to the directory in which the dataset is stored.
    :param img_name: str, the name of the image for which we want to gather eye-tracking data.
                     This must be of the form img.trn.xxx or img.tst.xxx
    :return list_all_files: list, containing the paths to all the relevant eye-tracking files.
    """
    list_all_files = glob.glob(os.path.join(dir_path, 'Eyetracker_Data/**/*' + img_name + '.*'), recursive=True)
    assert list_all_files, "List of eye-tracking files seem to be empty. Check directory or image name."
    return list_all_files

Now that we have our files, we can start having fun with the data itself.

#### 1) Fill the following function, creating a fixation map based on raw eye-tracking data.

In [ ]:
###### TODO ######
def create_fixmap(list_obs_files, img_w, img_h, factor_x, factor_y, t_begin=0, t_end=15):
    """
    Create a fixation map based on raw eye-tracking data.
    
    :param list_obs_files: list, a list of paths to the eye-tracking files of each observer for the considered image
    :param img_w: int, the width of the image, in pixels
    :param imw_h: int, the height of the image, in pixels
    :param factor_x: float, the ratio of the horizontal resolution to the horizontal size of the screen used in the eye-tracking experiment
    :param factor_y: float, the ratio of the vertical resolution to the vertical size of the screen used in the eye-tracking experimen
    :param t_begin: float, the start of the time slice to consider, in s.
    :param t_end: float, the end of the time slice to consider, in s.
    :return fixmap: numpy array, the fixation map.
    """
    # First, let's initialize the fixation map.
    fixmap = np.zeros((img_h, img_w))
    
    for obs in list_obs_files:
        # We will read the files and store them in a pandas dataframe.
        # If you are not familiar with pandas : https://pandas.pydata.org/docs/user_guide/index.html
        df = pd.read_csv(obs, sep=',', skiprows=19)
        # The 19 first lines of each file are just calibration data. We won't take them into account in this lab.
        
        # Each observer file represents 15s of viewing.
        # However, we might only be interested in a particular time slice of the data, so we'll filter the dataset using
        # the input time parameters
        
        ###### TODO ######
        
        # Let's also remove all lines where the eye was not tracked (e.g. because of blinks or tracking errors)

        ###### TODO ######
        
        # Now, we want to aggregate all poits belonging to the same fixation into a single point
        # For this, we will create groups of consecutive "Fixation" values, with a little bit of pandas magic !
        fix_groups = df.groupby((df["Fixation"].shift() != df["Fixation"]).cumsum())
        for n_group , group in fix_groups:
            if group["Fixation"].all() != 0:
                # For each detected fixation, we average the location of all gaze points
                x = group["ScreenPositionXmm"].mean() * 0.9
                y = -group["ScreenPositionYmm"].mean() * 0.9
                # Don't worry about the 0.9 coefficient; it's a fix due to errors in the experimental protocol
                # The minus sign before the Y coordinate is due to an inversion in the data, that used to be processed
                # in Matlab.
                # We then find the location of the fixation on the resulting fixation map

                ###### TODO ######
                
                # And we can finally add the fixation to the map
                
                ###### TODO ######
                
    return fixmap

#### 2) Now for a bit of geometry. Create the function compute_ppda, that computes the number of pixels per degree of visual angle based on the experimental conditions.

In [ ]:
###### TODO ######
def compute_ppda(distance, h_res, v_res, screen_w, screen_h):
    """
    Compute the number of pixels per degree of visual angle based on the experimental conditions.
    
    :param distance: int, the distance between the observer and the screen (in mm)
    :param h_res: int, the horizontal resolution of the screen
    :param v_res: int, the vertical resolution of the screen
    :param screen_w: int, the width of the screen (in mm)
    :param screen_h: int, the height of the screen (in mm)
    :return horizontal_ppda: float, the number of pixel per degree of visual angle
    """    
    ###### TODO ######
    
    return horizontal_ppda

#### 3) Create a function salmap_from_fixmap generating the visual saliency map based on the fixation map, and the number of pixels per degree of visual angle

In [ ]:
###### TODO ######
def salmap_from_fixmap(fixmap, ppda):
    """
    Generate a visual saliency map, based on the fixation map.
    
    :param fixmap: numpy array, the fixation map
    :param ppda: float, the number of pixels per degree of visual angle
    :return salmap: numpy array, the visual saliency map
    """

    ###### TODO ######
    return salmap

Now, let's visualize the saliency map we generated !

In [ ]:
list_img1 = get_files_all_observers(PATH_DATA, 'img.trn.1')
fixmap = create_fixmap(list_img1, 481, 321, FACTOR_X, FACTOR_Y)
img = mpimg.imread(os.path.join(PATH_DATA, "Images\\img.trn.1.png"))
# Values for the PPDA computation / image sizes / etc come from the experimental conditions.
# You can go read the associated paper for more information !
# J. Wang, D. M. Chandler, P. Le Callet, "Quantifying the relationship between visual salience and visual importance", Spie Human and Electronic imaging (HVEI) XV, San Jose, 2010

ppda = compute_ppda(415.8, 1280, 1024, 338, 270)
salmap = salmap_from_fixmap(fixmap, ppda)

In [ ]:
plt.imshow(img)

In [ ]:
plt.imshow(fixmap)

In [ ]:
plt.imshow(salmap)

#### 4) Now it's your turn ! Compute and display the saliency maps for a few images in the dataset. Explore it visually, and try to ensure that you haven't make mistakes while computing it. Vary the parameters to compute the map (assuming that the experimental condition changed); what can you say about it ? How does it influence the final representation, and why ?